In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time 
import datetime 

In [7]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


     |████████████████████████████████| 204.2MB 65kB/s 
     |████████████████████████████████| 204kB 44.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=772a802c7a3e29acdac15894526efed00564907a601c1dabe7a563ab5928a60a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


AttributeError: ignored

In [49]:
data = spark.read.csv("file1.csv",header=True, inferSchema=True)

In [50]:
data.show()

+---+----------+------------------+---------------------+-------+--------------+-------------------+--------------+------------------+---------------+-------------------+--------------------+-----------------+------------+---------+----------+----------+
|_c0|      date|     txVolume(USD)|adjustedTxVolume(USD)|txCount|marketcap(USD)|exchangeVolume(USD)|generatedCoins|              fees|activeAddresses| medianTxValue(USD)|           medianFee|averageDifficulty|paymentCount|blockSize|blockCount|price(USD)|
+---+----------+------------------+---------------------+-------+--------------+-------------------+--------------+------------------+---------------+-------------------+--------------------+-----------------+------------+---------+----------+----------+
|  8|2015-08-07|     5684487.22712|        5684487.22712| 2050.0|           0.0|           164329.0|   27437.65625|     37.3184135113|         1086.0| 41.611712872199995|    0.00132077886624| 1.47083865066E12|      1977.0|  3282693|   

In [51]:
feature_columns = data.columns[1:-1] # here we omit the final column
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns,outputCol="features")

In [52]:
from pyspark.sql.functions import col, unix_timestamp, to_date,to_timestamp


data = data.withColumn('date', to_timestamp(unix_timestamp(col('date'), 'yyyy-MM-dd').cast("timestamp")))


In [53]:
data = data.withColumn('date', (unix_timestamp(col('date'), 'yyyy-MM-dd').cast("int")))

In [54]:
data.show()

+---+----------+------------------+---------------------+-------+--------------+-------------------+--------------+------------------+---------------+-------------------+--------------------+-----------------+------------+---------+----------+----------+
|_c0|      date|     txVolume(USD)|adjustedTxVolume(USD)|txCount|marketcap(USD)|exchangeVolume(USD)|generatedCoins|              fees|activeAddresses| medianTxValue(USD)|           medianFee|averageDifficulty|paymentCount|blockSize|blockCount|price(USD)|
+---+----------+------------------+---------------------+-------+--------------+-------------------+--------------+------------------+---------------+-------------------+--------------------+-----------------+------------+---------+----------+----------+
|  8|1438905600|     5684487.22712|        5684487.22712| 2050.0|           0.0|           164329.0|   27437.65625|     37.3184135113|         1086.0| 41.611712872199995|    0.00132077886624| 1.47083865066E12|      1977.0|  3282693|   

In [55]:
data_2 = assembler.transform(data)

In [56]:
train, test = data_2.randomSplit([0.7, 0.3])

In [57]:
from pyspark.ml.regression import LinearRegression
algo = LinearRegression(featuresCol="features", labelCol="price(USD)")

In [58]:
model = algo.fit(train)

In [59]:
evaluation_summary = model.evaluate(test)

In [60]:
evaluation_summary.meanAbsoluteError

2.0224342814547174

In [61]:
evaluation_summary.rootMeanSquaredError

2.9760878109976536

In [62]:
evaluation_summary.r2

0.9998763658117484